In [1]:
import Post_Donors_PreProcess

In [2]:
import GPy
import GPyOpt
import numpy as np

In [3]:
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score

In [4]:
X, y, df_cols = Post_Donors_PreProcess.Post_Donor_PrePro(
                            Tf_Features=100, 
                            Sample=True, 
                            One_Hot=True, 
                            Standard_Scale=True)

DataFrame Init
Merging Teacher Information
Merging School Information
Merging Resource Managment
Compressing Data
Data Usage - change:Index                              0
Project Cost                   -5790
School Percentage Free Lunch   -5790
Project num Unique Resources   -5790
Total Resource Quantity        -5790
Mean Resource Cost             -5790
Total Project Cost             -5790
Median Resource Cost           -5790
Most exp Resource Cost         -5790
Least exp Resource Cost        -5790
Project Type                       0
Project Posted Year                0
Project Posted Month               0
Project Grade Level Category       0
Project Resource Category          0
Teacher Prefix                     0
School Metro Type                  0
School State                       0
School District                    0
School Zip                         0
dtype: int64
Total Change: -52110
Preprocessing
Text Processing


In [5]:
lgbm_train_data = lgbm.Dataset(data=X#.drop('School Percentage Free Lunch',axis=1)
                               ,label=y)

In [6]:
y.value_counts(normalize=True)

1    0.740217
0    0.259783
Name: Project Current Status, dtype: float64

## Default

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [9]:
model = lgbm.LGBMClassifier()
model.fit(X_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1)

In [10]:
model.score(X_test,y_test)

 /Users/Nahel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


0.7260869565217392

## Random Search

In [11]:
from sklearn.model_selection import RandomizedSearchCV

In [21]:
model = lgbm.LGBMClassifier(
    boosting_type = 'dart',
    application= 'binary',
    learning_rate= 0.01,
    scale_pos_weight= 2,
    drop_rate= 0.2,)

In [22]:
params_rand = {'num_leaves': [x for x in range(10,31,1)],
             'max_depth': [x for x in range(10,40)],
             'feature_fraction': [x / 1000.0 for x in range(820,900,20)],
             'bagging_fraction': [x / 1000.0 for x in range(820,900,20)],
             'lambda_l2' : [x / 100.0 for x in range(0,310,10)],

             'boosting_type': ['dart','gbdt']}

In [23]:
rs = RandomizedSearchCV(
    model, 
    params_rand, 
    n_iter=5,
    #scoring='roc_auc',
    cv=5)

rs.fit(X,y)

 /Users/Nahel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 /Users/Nahel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 /Users/Nahel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 /Users/Nahel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:The truth value of an empty array is ambiguous. Returning False, but in

 /Users/Nahel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 /Users/Nahel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 /Users/Nahel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 /Users/Nahel/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:The truth value of an empty array is ambiguous. Returning False, but in

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=LGBMClassifier(application='binary', boosting_type='dart', class_weight=None,
        colsample_bytree=1.0, drop_rate=0.2, learning_rate=0.01,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        scale_pos_weight=2, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1),
          fit_params=None, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'num_leaves': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], 'feature_fraction': [0.82, 0.84, 0.86, 0.88]...1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0], 'boosting_type': ['dart', 'gbdt'

In [24]:
rs.best_score_

0.7369565217391304

In [25]:
rs.best_params_

{'bagging_fraction': 0.84,
 'boosting_type': 'dart',
 'feature_fraction': 0.86,
 'lambda_l2': 0.9,
 'max_depth': 29,
 'num_leaves': 15}

## Bay Search

In [26]:
params_bay = [{'name': 'num_leaves',      'type': 'discrete',   'domain': (10,50)},
              {'name': 'max_depth' ,      'type': 'discrete',   'domain': (10,50)},
              {'name': 'feature_fraction','type': 'continuous', 'domain': (0.05,0.9)},
              {'name': 'bagging_fraction','type': 'continuous', 'domain': (0.05,0.9)},
              {'name': 'lambda_l2',       'type': 'continuous', 'domain': (0.0001,0.15)},]

In [27]:
def Bay_Wrapper(x):
    """
    Model Wrapper for Bay Opt 
    Wrapper used for LGBM & Bay:
    x[0][:] = list of all params for this aq
    """
    AUC = True
    if AUC: metric = 'auc'
    else: metric = 'binary_logloss'
    
    lgbm_params = { 'boosting': 'dart',
                    'application': 'binary',
                    'learning_rate': 0.01,
                    'scale_pos_weight': 2,
                    'drop_rate': 0.2,
                    'num_leaves': int(x[0][0]),
                    'max_depth': int(x[0][1]),
                    'feature_fraction': x[0][2],
                    'bagging_fraction': x[0][3],
                    'lambda_l2' : x[0][4],}

    cv_results = lgbm.cv(train_set=lgbm_train_data,
                         params=lgbm_params, 
                         nfold=5,
                         num_boost_round=600,
                         early_stopping_rounds=50,
                         metrics=[metric])

    optimum_boost_rounds = np.argmax(cv_results[metric+'-mean'])
    print('{} : {}'.format(metric,np.max(cv_results['auc-mean'])))
    
    return np.max(cv_results[metric+'-mean'])

In [ ]:
bay = GPyOpt.methods.BayesianOptimization(Bay_Wrapper,
                                          params_bay,
                                          acquisition_type ='EI',   # LCB acquisition
                                          acquisition_weight = .5,  # Exploration exploitation
                                          maximize=True)
bay.run_optimization(0) #GPyOpt uses 5 points + the number listed
print('Function after 5 Aquisitions')
bay.plot_acquisition()

In [ ]:
bay.run_optimization(90)
bay.plot_convergence()

In [ ]:
bay_pred = bay.x_opt
print('num_leaves : '+str(int(bay_pred[0])))
print('max_depth: '+str(int(bay_pred[1])))
print('feature_fraction: '+str(round(bay_pred[2],5)))
print('bagging_fraction: '+str(round(bay_pred[3],5)))
print('lambda_l2: '+str(round(bay_pred[4],5)))

In [ ]:
lgbm_params = { 'boosting': 'dart',
                'application': 'binary',
                'learning_rate': 0.01,
                'scale_pos_weight': 2,
                'drop_rate': 0.2,
                'num_leaves': int(bay_pred[0]),
                'max_depth': int(bay_pred[1]),
                'feature_fraction': bay_pred[2],
                'bagging_fraction': bay_pred[3],
                'lambda_l2' : bay_pred[4]}

cv_results = lgbm.cv(train_set=lgbm_train_data,
                     params=lgbm_params, 
                     nfold=5,
                     num_boost_round=600,
                     early_stopping_rounds=50,
                     #metrics=['accuracy']
                    )
#'binary_logloss-mean'
optimum_boost_rounds = np.argmax(cv_results['binary_logloss-mean'])
print('Auc : {}'.format(np.max(cv_results['binary_logloss-mean'])))